# Multilayer Tidal Calculations
In this cookbook we will explore how we can use TidalPy's multilayer tidal functions to calculate tidal heating as a function of radius.
Here we are using a propagation matrix technique (SVC16) that is only valid in the incompressible limit.

**References**:
- SVC16 : Sabadini, Vermeerson, & Cambiotti (2016, DOI: [10.1007/978-94-017-7552-6](https://doi.org/10.1007/978-94-017-7552-6).
- HH14  : Henning & Hurford (2014, DOI: [10.1088/0004-637X/789/1/30](https://doi.org/10.1088/0004-637X/789/1/30)).
- TB05  : Tobie et al. (2005), DOI: [10.1016/j.icarus.2005.04.006](https://doi.org/10.1016/j.icarus.2005.04.006).
- ID    : [IcyDwarf Code](https://github.com/MarcNeveu/IcyDwarf/blob/master/IcyDwarf/Thermal.h) written by Marc Neveu

## Build the planet

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl

from TidalPy import build_world
from TidalPy.tools.conversions import orbital_motion2semi_a
from TidalPy.utilities.numpy_helper import find_nearest
# For this example we will use the Io-Jupiter system
io = build_world('Io')
io.paint()
jupiter = build_world('Jupiter')
eccentricity = 0.0041
orbital_freq = 2. * np.pi / (86400. * 1.76914)
semi_major_axis = orbital_motion2semi_a(orbital_freq, jupiter.mass, io.mass)

# We won't be using TidalPy's OOP approach (which currently does not use the multi-layer approach,
#    instead relying on a homogenous model broken up by layers), instead we will pull out a few parameters needed
#    for the multilayer calculations.
radius_array = io.radii
volume_array = io.volume_slices
# Depth array skips the first shell since most viscoelastic properties are not defined there.
depth_array = (io.radius - radius_array)[1:]
gravity_array = io.gravities
density_array = io.densities
pressure_array = io.pressures

# We will give the core and mantle a different viscoelastic state, but to do that we need to know what index
#   corresponds to the top of the core.
core_radius_cutoff = find_nearest(radius_array, io.core.radius)
core_radius_cutoff_real = core_radius_cutoff - 1
mantle_N = len(radius_array[core_radius_cutoff:])
core_N = len(radius_array[:core_radius_cutoff])
core_N_real = core_N - 1
total_N = len(radius_array)
total_N_real = total_N - 1

# Determine some global properties
world_radius = radius_array[-1]
surface_gravity = gravity_array[-1]
mantle_bulk_density = np.average(density_array[core_radius_cutoff:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
# Setup Io's viscoelastic state - this is not particularly accurate, just meant for demonstration purposes

# All of the viscoelastic properties are not defined at r=0, so we are skipping the inner-most shell. This is done through
#    the "len(radius_array) - 1"

# Tidal dissipation is not too sensitive to bulk modulus so we will keep it constant
bulk_moduli = 200.0e9 * np.ones(len(radius_array) - 1)
bulk_moduli[:core_radius_cutoff_real] = 800.0e9

# Shear modulus and viscosity is much more important, but to keep this example simple we will assume a
#   constant shear and an increasing viscosity towards the surface
shear_moduli = 50.0e9 * np.ones(len(radius_array) - 1)
shear_moduli[:core_radius_cutoff_real] = 80.0e9

viscosities = 1.e30 * np.ones(len(radius_array) - 1)
viscosities[core_radius_cutoff_real:] = np.logspace(23, 16, mantle_N)

# Now we can calculate the complex_compliance of the world. We will just use a Maxwell model here but several
#    others are available.
from TidalPy.rheology.complex_compliance.compliance_models import maxwell_array, sundberg_array
complex_compliances = sundberg_array(orbital_freq, shear_moduli**(-1), viscosities)
# We will specifically turn off tidal dissipation in the core by setting the imaginary portion of
#    complex compliance = 0
complex_compliances[:core_radius_cutoff_real] = \
    np.real(complex_compliances[:core_radius_cutoff_real]) + 1.e-50j * np.ones(core_N_real)
complex_shears = complex_compliances**(-1)

# Now let's see what this looks like
fig_visco, ax_visco = plt.subplots()
ax_shear = ax_visco.twiny()
ax_comp = ax_visco.twiny()
ax_comp.spines["top"].set_position(("axes", 1.2))

ax_visco.plot(viscosities, depth_array/1000., c='b', ls=':')
ax_shear.plot(shear_moduli, depth_array/1000., c='r', ls=':')
ax_comp.plot(np.real(complex_shears), depth_array/1000., c='orange', ls='-')
ax_comp.plot(np.imag(complex_shears), depth_array/1000., c='orange', ls='--')
ax_visco.set_ylim(ax_visco.get_ylim()[::-1])
ax_comp.set_ylim(ax_comp.get_ylim()[::-1])
ax_shear.set_ylim(ax_shear.get_ylim()[::-1])

ax_visco.set(ylabel='Depth [km]', xlabel='Viscosity [Pa s]', xscale='log')
ax_shear.set(xlabel='Shear Modulus [Pa]', xscale='linear')
ax_comp.set(xlabel='Complex Shear (Solid=Real, Dash=Imag) [Pa$^{-1}$]', xscale='log')
for ax, color in zip([ax_visco, ax_shear, ax_comp], ['b', 'r', 'orange']):
    if ax is ax_visco:
        ax.spines['bottom'].set_color(color)
    else:
        ax.spines['top'].set_color(color)
    ax.xaxis.label.set_color(color)

fig_visco.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
np.seterr(divide='raise')
# Load TidalPy's multilayer functions
from TidalPy.tides.multilayer import fundamental_matrix_orderl2, propagate, decompose

# Find the fundamental matrix; skip the innermost shell as that will be a boundary condition.
Y, Y_inv, derivative_mtx = fundamental_matrix_orderl2(radius_array[1:], complex_shears, density_array[1:], gravity_array[1:])

In [4]:
# Propagate the tidal solution through the world's shells
central_boundary_condition = np.zeros((6, 3), dtype=np.complex128)
# Roberts & Nimmo (2000): Liquid innermost zone.
central_boundary_condition[2, 0] = 1.0
central_boundary_condition[3, 1] = 1.0
central_boundary_condition[5, 2] = 1.0

tidal_y, tidal_y_deriv = propagate(Y, Y_inv, derivative_mtx, central_boundary_condition, world_radius, order_l=2)

In [5]:
# Decompose the tidal solution into useful properties
#    There is a gradient in this function so we lose another shell. Thus the [1:] in viscoelastic properties
radial_sensitivity_to_shear, (k2, h2, l2) = \
    decompose(tidal_y, tidal_y_deriv, radius_array[1:], gravity_array[1:], complex_shears, bulk_moduli, order_l=2)

print('Surface Love & Shida Numbers')
print(f'k_2 = {k2[-1]:.2e}; h_2 = {h2[-1]:.2e}; l_2 = {l2[-1]:.2e}')

# Let's plot these as a function of depth.
fig_love, love_axes = plt.subplots(ncols=2)
fig_love.suptitle('Love and Shida Numbers')

for ax_i, ax in enumerate(love_axes):
    ax_love_h = ax.twiny()
    ax_love_l = ax.twiny()
    ax_love_l.spines["top"].set_position(("axes", 1.2))
    
    if ax_i == 0:
        ax.plot(np.real(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(np.real(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(np.real(l2), depth_array/1000., c='orange', ls='-')
        
        ax.set(ylabel='Depth [km]', xlabel='Re[$k_{2}$]', xscale='linear')
        ax_love_h.set(xlabel='Re[$h_{2}$]', xscale='linear')
        ax_love_l.set(xlabel='Re[$l_{2}$]', xscale='linear')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])
    else:
        ax.plot(-np.imag(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(-np.imag(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(-np.imag(l2), depth_array/1000., c='orange', ls='-')

        ax.set(ylabel='Depth [km]', xlabel='-Im[$k_{2}$]', xscale='log')
        ax_love_h.set(xlabel='-Im[$h_{2}$]', xscale='log')
        ax_love_l.set(xlabel='-Im[$l_{2}$]', xscale='log')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])

    for ax2, color in zip([ax, ax_love_h, ax_love_l], ['b', 'r', 'orange']):
        if ax2 is ax_visco:
            ax2.spines['bottom'].set_color(color)
        else:
            ax2.spines['top'].set_color(color)
        ax2.xaxis.label.set_color(color)

fig_love.tight_layout()
plt.show()

Surface Love & Shida Numbers
k_2 = -2.05e+00+9.88e-03j; h_2 = 8.81e-02-1.79e-02j; l_2 = 2.64e-02-5.38e-03j


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Calculate Tidal Displacement
Utilizing the simplified tidal potential (low eccentricity, no obliquity, spin-synchronous).

In [6]:
from TidalPy.tides.potential.synchronous_low_e import tidal_potential

# Define latitude and longitude domain
colatitude = np.linspace(0.1, 179.9, 70)
latitude = colatitude - 90.
longitude = np.linspace(0., 360., 75)
long_mtx, lat_mtx = np.meshgrid(np.radians(longitude), np.radians(colatitude))

displacements_over_time = []

for time in np.linspace(0., 86400. * 2., 20):
    # Calculate the simplified tidal potential
    potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
        tidal_potential(world_radius, long_mtx, lat_mtx, orbital_freq, eccentricity, time)

    # Scale by the tidal solutions
    radial_displacement = tidal_y[0, -1] * potential
    polar_displacement = tidal_y[2, -1] * potential_dtheta
    azimuthal_displacement = tidal_y[2, -1] * potential_dphi / np.sin(lat_mtx)
    
    displacements_over_time.append((time / (60. * 60), radial_displacement, polar_displacement, azimuthal_displacement))

In [7]:
# Plot results
from ipywidgets import interact
fig_disp, axes_disp = plt.subplots(ncols=3, figsize=(8, 4))
plt.subplots_adjust(wspace=.5)
ax_r = axes_disp[0]
ax_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Radial Displacement', xlim=(0, 360), ylim=(-90, 90))
ax_th = axes_disp[1]
ax_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Polar Displacement', xlim=(0, 360), ylim=(-90, 90))
ax_phi = axes_disp[2]
ax_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Azimuthal Displacement', xlim=(0, 360), ylim=(-90, 90))
fig_disp.suptitle(f'0.00 Hour')

time_hr, u_r, u_th, u_phi = displacements_over_time[0]
ax_r.contourf(longitude, latitude, u_r, 40)
ax_th.contourf(longitude, latitude, u_th, 40)
ax_phi.contourf(longitude, latitude, u_phi, 40)

def update_fig(time_i=0):
    time_hr, u_r, u_th, u_phi = displacements_over_time[time_i]
    ax_r.clear()
    ax_th.clear()
    ax_phi.clear()
    fig_disp.suptitle(f'{time_hr:0.2f} Hour')
    ax_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Radial Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Polar Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Azimuthal Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_r.contourf(longitude, latitude, u_r, 20)
    ax_th.contourf(longitude, latitude, u_th, 20)
    ax_phi.contourf(longitude, latitude, u_phi, 20)
    fig_disp.tight_layout()

interact(update_fig, time_i=(0, len(displacements_over_time)-1, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=19), Output()), _dom_classes=('widget-inter…

<function __main__.update_fig(time_i=0)>

## Orbit-Averaged, Tidal Heating as Function of Depth
Here we will use a modified version of the heating(radius) equations found in TB05. 

In [8]:
from TidalPy.tides.multilayer.heating import calc_radial_tidal_heating

# We will use the radial sensitivity to shear calculated in a previous step.
vol_heating_as_func_radius = \
    calc_radial_tidal_heating(eccentricity, orbital_freq, semi_major_axis,
                              jupiter.mass,
                              radius_array[1:], radial_sensitivity_to_shear,
                              complex_shears, order_l=2)

print(f'Total Heating {sum(vol_heating_as_func_radius * volume_array[1:] ):0.2e} W')
# Plot result
fig_heat, ax_heat = plt.subplots()
ax_heat.set(ylabel='Depth [km]', xlabel='Volumetric Heating Rate [W m$^{-3}$]', xscale='log')
ax_heat.plot(vol_heating_as_func_radius, depth_array / 1000.)
ax_heat.set_ylim(ax_heat.get_ylim()[::-1])
plt.show()

Total Heating 1.22e+13 W


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Strain and Stress Tensors
Here we will calculate the strain and stress tensors using the methods described in TB05 and B13

In [9]:
from TidalPy.tides.multilayer.stress_strain import calculate_strain

# Use the co-lat. and long. domain as setup in the displacements step

strain_tensor_over_time = list()

for time in np.linspace(0., 86400. * 2., 20):
    # Calculate the simplified tidal potential
    potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
        tidal_potential(world_radius, long_mtx, lat_mtx, orbital_freq, eccentricity, time)

    # Use the tidal solutions to scale the potential and find the strains
    strain_tensor = \
        calculate_strain(potential, potential_dtheta, potential_dphi,
                         potential_d2theta, potential_d2phi,
                         potential_dtheta_dphi, tidal_y, tidal_y_deriv,
                         colatitude=lat_mtx, radius_array=radius_array[1:], shear_moduli=complex_shears)
    
    strain_tensor_over_time.append((time / (60. * 60), strain_tensor))
    # Note, strain tensor flips the radius position (radius comes first)

### Stress over time

In [12]:
# Plot results
from ipywidgets import interact
fig_strain, axes_strain = plt.subplots(ncols=3, nrows=2, figsize=(8, 8))
plt.subplots_adjust(wspace=.5, hspace=0.3)
ax_r = axes_strain[0, 0]
ax_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{rr}$', xlim=(0, 360), ylim=(-90, 90))
ax_th = axes_strain[0, 1]
ax_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\theta\\theta}$', xlim=(0, 360), ylim=(-90, 90))
ax_phi = axes_strain[0, 2]
ax_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\phi\\phi}$', xlim=(0, 360), ylim=(-90, 90))
ax_rth = axes_strain[1, 0]
ax_rth.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{r\\theta}$', xlim=(0, 360), ylim=(-90, 90))
ax_rphi = axes_strain[1, 1]
ax_rphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{r\\phi}$', xlim=(0, 360), ylim=(-90, 90))
ax_thphi = axes_strain[1, 2]
ax_thphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\theta\\phi}$', xlim=(0, 360), ylim=(-90, 90))


fig_strain.suptitle(f'Surface Strains; 0.00 Hour')
time_hr, strains_by_r = strain_tensor_over_time[0]
strains_at_surf = strains_by_r[-1, :, :, :, :]
e_rr = strains_at_surf[0, 0, :, :]
e_thth = strains_at_surf[1, 1, :, :]
e_phph = strains_at_surf[2, 2, :, :]
e_rth = 2. * strains_at_surf[0, 1, :, :]
e_rph = 2. * strains_at_surf[0, 2, :, :]
e_thph = 2. * strains_at_surf[1, 2, :, :]
ax_r.contourf(longitude, latitude, e_rr, 40)
ax_th.contourf(longitude, latitude, e_thth, 40)
ax_phi.contourf(longitude, latitude, e_phph, 40)
ax_rth.contourf(longitude, latitude, e_rr, 40)
ax_rphi.contourf(longitude, latitude, e_thth, 40)
ax_thphi.contourf(longitude, latitude, e_phph, 40)

def update_fig(time_i=0):
    time_hr, strains_by_r = strain_tensor_over_time[time_i]
    strains_at_surf = strains_by_r[-1, :, :, :, :]
    e_rr = strains_at_surf[0, 0, :, :]
    e_thth = strains_at_surf[1, 1, :, :]
    e_phph = strains_at_surf[2, 2, :, :]
    e_rth = 2. * strains_at_surf[0, 1, :, :]
    e_rph = 2. * strains_at_surf[0, 2, :, :]
    e_thph = 2. * strains_at_surf[1, 2, :, :]
    
    ax_r.clear()
    ax_th.clear()
    ax_phi.clear()
    fig_strain.suptitle(f'Surface Strains; {time_hr:0.2f} Hour')
    ax_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{rr}$', xlim=(0, 360), ylim=(-90, 90))
    ax_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\theta\\theta}$', xlim=(0, 360), ylim=(-90, 90))
    ax_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\phi\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_rth.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{r\\theta}$', xlim=(0, 360), ylim=(-90, 90))
    ax_rphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{r\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_thphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='$\\epsilon_{\\theta\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_r.contourf(longitude, latitude, e_rr, 20)
    ax_th.contourf(longitude, latitude, e_thth, 20)
    ax_phi.contourf(longitude, latitude, e_phph, 20)
    ax_rth.contourf(longitude, latitude, e_rth, 20)
    ax_rphi.contourf(longitude, latitude, e_rph, 20)
    ax_thphi.contourf(longitude, latitude, e_thph, 20)
    
    fig_strain.tight_layout()

interact(update_fig, time_i=(0, len(strain_tensor_over_time)-1, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=19), Output()), _dom_classes=('widget-inter…

<function __main__.update_fig(time_i=0)>

### Stress over depth
**(colat = 90, long = 0)**

In [18]:
colatitude = 90.
longitude = 0.


# Calculate the simplified tidal potential
potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
    tidal_potential(world_radius, np.radians(longitude), np.radians(colatitude), orbital_freq, eccentricity, time=0.0)

# Use the tidal solutions to scale the potential and find the strains
strain_tensor_by_r = \
    calculate_strain(potential, potential_dtheta, potential_dphi,
                     potential_d2theta, potential_d2phi,
                     potential_dtheta_dphi, tidal_y, tidal_y_deriv,
                     colatitude=np.radians(colatitude), radius_array=radius_array[1:], shear_moduli=complex_shears)

# Plot results
fig_strain, axes_strain = plt.subplots(ncols=3, nrows=2, figsize=(8, 8))
plt.subplots_adjust(wspace=.5, hspace=0.3)
ax_r = axes_strain[0, 0]
ax_r.set(ylabel='Depth [km]', title='$\\epsilon_{rr}$')
ax_th = axes_strain[0, 1]
ax_th.set(title='$\\epsilon_{\\theta\\theta}$')
ax_phi = axes_strain[0, 2]
ax_phi.set(title='$\\epsilon_{\\phi\\phi}$')
ax_rth = axes_strain[1, 0]
ax_rth.set(ylabel='Depth [km]', title='$\\epsilon_{r\\theta}$')
ax_rphi = axes_strain[1, 1]
ax_rphi.set(title='$\\epsilon_{r\\phi}$')
ax_thphi = axes_strain[1, 2]
ax_thphi.set(title='$\\epsilon_{\\theta\\phi}$')

e_rr = strain_tensor_by_r[:, 0, 0]
e_thth = strain_tensor_by_r[:, 1, 1]
e_phph = strain_tensor_by_r[:, 2, 2]
e_rth = 2. * strain_tensor_by_r[:, 0, 1]
e_rph = 2. * strain_tensor_by_r[:, 0, 2]
e_thph = 2. * strain_tensor_by_r[:, 1, 2]

ax_r.plot(e_rr, depth_array / 1000., 'k-')
ax_r.set_ylim(ax_r.get_ylim()[::-1])
ax_th.plot(e_thth, depth_array / 1000., 'k-')
ax_th.set_ylim(ax_th.get_ylim()[::-1])
ax_phi.plot(e_phph, depth_array / 1000., 'k-')
ax_phi.set_ylim(ax_phi.get_ylim()[::-1])
ax_rth.plot(e_rth, depth_array / 1000., 'k-')
ax_rth.set_ylim(ax_rth.get_ylim()[::-1])
ax_rphi.plot(e_rph, depth_array / 1000., 'k-')
ax_rphi.set_ylim(ax_rphi.get_ylim()[::-1])
ax_thphi.plot(e_thph, depth_array / 1000., 'k-')
ax_thphi.set_ylim(ax_thphi.get_ylim()[::-1])

fig_strain.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Performance
Here we are just doing a simple check on how the propagation functions perform

In [11]:
run_performance = False

if run_performance:
    print('Building Matricies')
    %timeit fundamental_matrix_orderl2(radius_array[1:], complex_shears[1:], density_array[1:], gravity_array[1:])

    print('Propagating Solutions')
    %timeit propagate(Y, Y_inv, central_boundary_condition, world_radius, order_l=2)

    print('Decomposition')
    %timeit decompose(tidal_y, radius_array[1:], gravity_array[1:], complex_shears[2:], bulk_moduli[2:], order_l=2)